# Not Working Yet

In [1]:
# !uv pip install cmfrec implicit

In [7]:
import cmfrec
import numpy as np
from scipy.sparse import csr_matrix
from load_interactions import load_interactions
from temporal_split import create_temporal_split
from evaluate import evaluate_recommendations

# Load and split data
likes_df, follows_df, posts_df = load_interactions()
train_data, test_data, test_interactions = create_temporal_split(likes_df)

# Convert training data to sparse matrix format
# Use the same user/post mappings as in train_data
user2id = train_data.user2id
post2id = train_data.post2id

# Create sparse matrix from training data
rows = []
cols = []
for _, row in likes_df.iterrows():
    if row['user_uri'] in user2id and row['post_uri'] in post2id:
        rows.append(user2id[row['user_uri']])
        cols.append(post2id[row['post_uri']] - len(user2id))  # Adjust post IDs to start from 0

data = np.ones(len(rows))  # Use 1s for implicit feedback
train_matrix = csr_matrix((data, (rows, cols)), 
                         shape=(len(user2id), len(post2id)))

# Train MF model
model = cmfrec.CMF(
    method='als',  # Alternating Least Squares
    factors=64,    # Same embedding dimension as LightGCN
    lambda_=1.0,   # Regularization parameter
    alpha=15.0,    # Confidence scaling factor for implicit feedback
    niter=15       # Number of iterations
)

# Fit the model
model.fit(X=train_matrix)

# Function to evaluate MF model using same metrics
def evaluate_mf_model(model, test_interactions, k_values=[20, 100, 1000]):
    recalls = {k: [] for k in k_values}
    
    for user_idx, true_post_ids in test_interactions.items():
        # Get recommendations for user
        # Note: model.topN returns (scores, indices)
        recs = model.topN(user=user_idx, n=max(k_values))[1]
        true_set = set(p - len(user2id) for p in true_post_ids)  # Adjust post IDs
        
        # Calculate recall@k for each k
        for k in k_values:
            rec_set = set(recs[:k])
            recall = len(rec_set.intersection(true_set)) / len(true_set) if true_set else 0
            recalls[k].append(recall)
    
    # Calculate average recall for each K
    avg_recalls = {k: sum(scores)/len(scores) for k, scores in recalls.items()}
    return avg_recalls

# Evaluate MF model
mf_results = evaluate_mf_model(model, test_interactions)
print("\nMatrix Factorization Results:")
for k, recall in mf_results.items():
    print(f"Recall@{k}: {recall:.4f}")

Loaded 3618997 likes, 2426775 follows, and 3618975 unique posts before May 2023
Number of unique users: 30470, number of unique posts: 816056
Number of unique users: 7062, number of unique posts: 15891
Train interactions: 2895197
Test interactions: 38186
Split timestamp: 2023-04-29 03:11:50.981000
Users with test interactions: 7062


TypeError: CMF.__init__() got an unexpected keyword argument 'factors'